In [ ]:
import cv2
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm.notebook import tqdm
import random
from pathlib import Path
import json
import lensfunpy
from typing import Tuple

In [ ]:
ROOT_DIR = os.path.dirname(os.getcwd())
DATA_FOLDER = os.path.join(ROOT_DIR, "data")

In [ ]:
def calibrate_camera_from_video(
    path_to_video: str, 
    checkerboard: Tuple[int, int], 
    n_images: int = 500,
    use_LU: bool = False,
    verbose: bool = False, 
    show_successful: bool = False,
    show_failed: bool = False
):
    CHECKERBOARD = checkerboard
    processed_images = 0
    three_d_points = []
    two_d_points = []
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    video_capture = cv2.VideoCapture(path_to_video)
    n_frames = video_capture.get(cv2.CAP_PROP_FRAME_COUNT)
    objectp3d = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
    objectp3d[0, :, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
    image_shape = None
    
    if verbose is True:
        print(f"Collecting images and points")
    
    with tqdm(total = n_images) as pbar:    
        while processed_images < n_images:

            random_frame_number = random.randint(0, n_frames)
            video_capture.set(cv2.CAP_PROP_POS_FRAMES, random_frame_number)
            success, image = video_capture.read()
            if not success:
                print(f"Reading frame number {random_frame_number} failed, retrying")
                continue

            image_shape = image.shape[0:2]

            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Find the chess board corners
            # If desired number of corners are
            # found in the image then ret = true
            ret, corners = cv2.findChessboardCorners(gray_image, CHECKERBOARD, 
                                                     cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
            # If desired number of corners can be detected then,
            # refine the pixel coordinates and display
            # them on the images of checker board
            if ret == True:
                three_d_points.append(objectp3d)
                # Refining pixel coordinates
                # for given 2d points.
                corners2 = cv2.cornerSubPix(gray_image, corners, (11, 11), (-1, -1), criteria)
                two_d_points.append(corners2)
                
                processed_images += 1
                pbar.update(1)
            
                if show_successful is True:
                    image = cv2.drawChessboardCorners(image, CHECKERBOARD, corners2, ret)
                    plt.imshow(image)
                    plt.show()

            else:
                if verbose is True:
                    print(f"Pattern was not found on image {filename}")
                    
                if show_failed is True:
                    image = cv2.drawChessboardCorners(image, CHECKERBOARD, corners, ret)
                    plt.imshow(image)
                    plt.show()
           

    if verbose is True:
        print("Calculating calibration matrix")
        
    if use_LU is True:
        ret, intrinsic_matrix, distortion, r_vecs, t_vecs = cv2.calibrateCamera(
            three_d_points, two_d_points, image_shape[::-1], None, None, flags = cv2.CALIB_USE_LU
        )
    else:
        ret, intrinsic_matrix, distortion, r_vecs, t_vecs = cv2.calibrateCamera(
            three_d_points, two_d_points, image_shape[::-1], None, None
        )
    if ret == False:
        print(f"Camera calibration failed")
        return None
    

    print("Camera matrix: ")
    print(intrinsic_matrix)
    print("Distortion coefficient: ")
    print(distortion)
    
    projection_error = 0
    for i in range(len(three_d_points)):
        img_points2, _ = cv2.projectPoints(three_d_points[i], r_vecs[i], t_vecs[i], intrinsic_matrix, distortion)
        error = cv2.norm(two_d_points[i], img_points2, cv2.NORM_L2) / len(img_points2)
        projection_error += error

    print(f"Total projection error: {projection_error}")
    
    return intrinsic_matrix, distortion, r_vecs, t_vecs, projection_error


In [ ]:
def save_camera_calibration_files(file_prefix: str, output_path: str, intrinsic_matrix, distortion, r_vecs, t_vecs):
    intrinsic_matrix_path = os.path.join(output_path, f"{file_prefix}_intrinsic_matrix.npy")
    camera_distortion_path = os.path.join(output_path, f"{file_prefix}_camera_distortion.npy")
    radial_vector_path = os.path.join(output_path, f"{file_prefix}_radial_distortion_coef.npy")
    tangential_vector_path = os.path.join(output_path, f"{file_prefix}_tangential_distortion_coef.npy")

    np.save(intrinsic_matrix_path, intrinsic_matrix)
    np.save(camera_distortion_path, distortion)
    np.save(radial_vector_path, r_vecs)
    np.save(tangential_vector_path, t_vecs)

In [ ]:
# Iterate over all calibration folders and calculate calibration matrix for each folder

calibration_videos_path = os.path.join(DATA_FOLDER, "camera_calibration", "go_pro_5_hero_black")
projection_errors = {}
for filename in tqdm(glob.iglob(f"{calibration_videos_path}/**/*.MP4", recursive=True)):
    if os.path.isfile(filename):
        print(f"Processing file {filename}")
        path_parts = Path(filename).parts
        board_folder = path_parts[10]
        checkboard_size = tuple(list(map(lambda x: x - 1, list(map(int, board_folder.split("_")[1].split("x"))))))

        output_file_prefix = "_".join(path_parts[9:12])
        output_path = os.path.join(*path_parts[0:10])
        
        intrinsic_matrix, distortion, r_vecs, t_vecs, proj_error = calibrate_camera_from_video(
            path_to_video = filename, 
            checkerboard = checkboard_size,
            n_images = 200,
            use_LU = True,
            verbose = True,
            show_failed = False
        )
        
        projection_errors[output_file_prefix] = proj_error
        # Sort projection errors
        projection_errors = dict(sorted(projection_errors.items(), key=lambda item: item[1]))

        save_camera_calibration_files(output_file_prefix, output_path, intrinsic_matrix, distortion, r_vecs, t_vecs)
        
        projection_errors_file_path = os.path.join(calibration_videos_path, "projection_errors.json")
        with open(projection_errors_file_path, 'w') as file:
            file.write(json.dumps(projection_errors))
        
projection_errors_sorted = dict(sorted(projection_errors.items(), key=lambda item: item[1]))
print("Project errors:")
print("==============\n")
for key, value in projection_errors_sorted.items():
    print(f"{key}: {value} \n")

In [ ]:
best_matrix = "camera_1_checkerboard_8x10_20x20_slow"

calibration_data_folder = os.path.join(DATA_FOLDER, "camera_calibration", "go_pro_5_hero_black")

intinsic_matrix = np.load(
    os.path.join(
        calibration_data_folder, "camera_1", f"{best_matrix}_intrinsic_matrix.npy"
    )
)
distorion = np.load(
    os.path.join(
        calibration_data_folder, "camera_1", f"{best_matrix}_camera_distortion.npy"
    )
)

In [ ]:
h, w = 1080, 1920
undistorted_matrix, roi = cv2.getOptimalNewCameraMatrix(intrinsic_matrix, distortion, (w,h), 1, (w,h))

keparoi_image_path = os.path.join(DATA_FOLDER, "keparoi_right_frame.jpg")
image = cv2.imread(keparoi_image_path)
undistorted_image = cv2.undistort(image, intrinsic_matrix, distortion, None, undistorted_matrix)

x, y, w, h = roi
undistorted_image = undistorted_image[y:y+h, x:x+w]
plt.figure(figsize=(30,50))
plt.subplot(1,2,1)
plt.imshow(image)
plt.subplot(1,2,2)
plt.imshow(undistorted_image)
plt.show()